In [2]:
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import dask.array as da
import intake

import requests

In [3]:
pd.options.display.max_columns = 999

In [38]:
claims_df = pd.read_csv("data/claims.csv")

claims_features= claims_df.loc[:, ("dateOfLoss", 
                          "baseFloodElevation", 
                          "ratedFloodZone", 
                          "locationOfContents", 
                          "occupancyType", 
                          "amountPaidOnBuildingClaim",
                          "amountPaidOnContentsClaim",
                          "totalBuildingInsuranceCoverage",
                          "totalContentsInsuranceCoverage",
                          "yearOfLoss",
                          "primaryResidenceIndicator",
                          "buildingDamageAmount", 
                          "netBuildingPaymentAmount",
                          "buildingPropertyValue",
                          "contentsDamageAmount",
                          "netContentsPaymentAmount",
                          "contentsPropertyValue",
                          "floodCharacteristicsIndicator",
                          "floodWaterDuration",
                          "floodproofedIndicator",
                          "floodEvent",
                          "buildingReplacementCost",
                          "contentsReplacementCost",
                          "stateOwnedIndicator",
                          "buildingDescriptionCode",
                          "rentalPropertyIndicator",
                          "state",
                          "countyCode",
                          "censusTract",
                          "censusBlockGroupFips",
                          "latitude",
                          "longitude",
                          "id")]

claims_features['countyCode'] = claims_features['countyCode'].astype(str).str.rstrip('.0')

claims_features['censusTract'] = claims_features['censusTract'].astype(str).str.rstrip('.0')

claims_features['censusBlockGroupFips'] = claims_features['censusBlockGroupFips'].astype(str).str.rstrip('.0')



# Remove rows with missing geometry
claims_features = claims_features.dropna(subset=["latitude", "longitude"])


# Create our GeoDataFrame with geometry column created from lon/lat
claims = gpd.GeoDataFrame(
    claims_features,
    geometry=gpd.points_from_xy(claims_df["longitude"], claims_df["latitude"]),
    crs="EPSG:4326",
)

claims_gpd = claims.to_crs('EPSG:3857')

claims_gpd['x'] = claims_gpd['geometry'].x
claims_gpd['y'] = claims_gpd['geometry'].y

#claims_gpd['yearBuilt'] = claims_gpd['originalConstructionDate'].str[:4]


C:\Users\dtdre\AppData\Local\Temp\ipykernel_17076\3049984955.py:1: DtypeWarning: Columns (8,36,39,43,52,53,62) have mixed types. Specify dtype option on import or set low_memory=False.
  claims_df = pd.read_csv("data/claims.csv")


In [39]:
states = ['NJ']


claims_NJ = claims_gpd[(claims_gpd['yearOfLoss'] >= 1995) & (claims_gpd['state'].isin(states))]

claims_NJ


dateOfLoss  baseFloodElevation ratedFloodZone  \
1        2012-10-29T00:00:00.000Z                 NaN             AE   
42606    2012-10-29T00:00:00.000Z                11.0             AE   
42607    2018-09-09T00:00:00.000Z                 NaN             AE   
42608    2022-10-02T00:00:00.000Z                 NaN             AE   
42609    2012-10-29T00:00:00.000Z                 NaN             AE   
...                           ...                 ...            ...   
2473602  1999-09-16T00:00:00.000Z                 NaN              X   
2473603  2012-10-29T00:00:00.000Z                 NaN            A10   
2473604  2007-04-15T00:00:00.000Z                 NaN            A10   
2473605  2011-08-28T00:00:00.000Z                 NaN              X   
2473606  2007-04-16T00:00:00.000Z                 NaN              X   

         locationOfContents  occupancyType  amountPaidOnBuildingClaim  \
1                       NaN            1.0                   55000.00   
42606                   7.0            1.0                   12979.08   
42607                   4.0            6.0                   62702.76   
42608                   4.0           18.0                   16801.05   
42609                   2.0            1.0                   17530.18   
...                     ...            ...                        ...   
2473602                 NaN            1.0                  163982.11   
2473603                 NaN            1.0                   13030.26   
2473604                 NaN            1.0                    7502.63   
2473605                 2.0            1.0                    1979.98   
2473606                 NaN            1.0                   20000.00   

         amountPaidOnContentsClaim  totalBuildingInsuranceCoverage  \
1                             0.00                         55000.0   
42606                      1630.00                         76200.0   
42607                      9628.74                        250000.0   
42608                         0.00                        250000.0   
42609                      5000.00                        120000.0   
...                            ...                             ...   
2473602                   34800.00                        250000.0   
2473603                       0.00                        250000.0   
2473604                       0.00                        200000.0   
2473605                       0.00                        194700.0   
2473606                    8000.00                         20000.0   

         totalContentsInsuranceCoverage  yearOfLoss  \
1                                   0.0        2012   
42606                            6400.0        2012   
42607                           30000.0        2018   
42608                           30000.0        2022   
42609                            5000.0        2012   
...                                 ...         ...   
2473602                         34800.0        1999   
2473603                             0.0        2012   
2473604                             0.0        2007   
2473605                         15800.0        2011   
2473606                          8000.0        2007   

         primaryResidenceIndicator  buildingDamageAmount  \
1                                1               67506.0   
42606                            1               10933.0   
42607                            0               64703.0   
42608                            0               18801.0   
42609                            1               19530.0   
...                            ...                   ...   
2473602                          0              164482.0   
2473603                          1               16031.0   
2473604                          1               10504.0   
2473605                          1                3980.0   
2473606                          1               22839.0   

         netBuildingPaymentAmount  buildingPropertyValue  

In [40]:
from pathlib import Path  

filepath = Path('data/claims_NJ.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

claims_NJ.to_csv(filepath) 